# Long Short Term Memory

Long Short Term Memory networks – usually just called “LSTMs” – are a special kind of RNN, capable of learning long-term dependencies. They were introduced by [Hochreiter & Schmidhuber (1997)](http://www.bioinf.jku.at/publications/older/2604.pdf), and were refined and popularized by many people in following work. They work tremendously well on a large variety of problems, and are now widely used.

At each time step, the LSTM cell takes in 3 different pieces of information the current input data, the short-term memory from the previous cell (similar to hidden states in RNNs) and lastly the long-term memory. The short-term memory is commonly referred to as the hidden state, and the long-term memory is usually known as the cell state.

The cell then uses gates to regulate the information to be kept or discarded at each time step before passing on the long-term and short-term information to the next cell.

These gates are called the Input Gate, the Forget Gate, and the Output Gate. The LSTM does have the ability to remove or add information to the cell state, carefully regulated by structures called gates.   
<font size="4">$f_t = \sigma(W_f.[h_{t-1}, x_t]+b_f)$</font>  
<font size="4">$i_t = \sigma(W_i.[h_{t-1}, x_t]+b_i)$</font>  
<font size="4">$\tilde{C}_t = tanh(W_c.[h_{t-1}, x_t]+b_c)$</font>  
<font size="4">$C_t = f_t*C_{t-1}+i_{t}*\tilde{C}_t$</font>  
<font size="4">$o_t = \sigma(W_o[h_{t-1}, x_t] + b_o)$</font>  
<font size="4">$h_t = o_t * tanh(C_t)$</font>    


### forget gate layer
The forget layer decides what information we’re going to throw away from the cell state. It looks at $h_{t−1}$ and $x_t$, and outputs a number between 0 and 1 for each number in the cell state $C_{t−1}$. 
![LSTM1](Images/LSTM1.png) 


The next step is to decide what new information we’re going to store in the cell state. This has two parts. First, a sigmoid layer called the **“input gate layer”** decides which values we’ll update.  
![LSTM2](Images/LSTM2.png)


It’s now time to update the old cell state, $C_{t−1}$, into the new cell state $C_t$. The previous steps already decided what to do, we just need to actually do it. 
We multiply the old state by $f_t$, forgetting the things we decided to forget earlier. Then we add $i_t∗\tilde{C}_t$. This is the new candidate values, scaled by how much we decided to update each state value.  
![LSTM3](Images/LSTM3.png)

Finally, we need to decide what we’re going to output. This output will be based on our cell state, but will be a filtered version. First, we run a sigmoid layer which decides what parts of the cell state we’re going to output. 
Then, we put the cell state through tanh (to push the values to be between −1 and 1) and multiply it by the output of the sigmoid gate, so that we only output the parts we decided to.  
![LSTM4](Images/LSTM4.png)

In [1]:
import torch
import torch.nn as nn

In [6]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.weight_ih = Parameter(torch.randn(4 * hidden_size, input_size))
        self.weight_hh = Parameter(torch.randn(4 * hidden_size, hidden_size))
        self.bias_ih = Parameter(torch.randn(4 * hidden_size))
        self.bias_hh = Parameter(torch.randn(4 * hidden_size))

    def forward(self, input, state):
        # type: (Tensor, Tuple[Tensor, Tensor]) -> Tuple[Tensor, Tuple[Tensor, Tensor]]
        hx, cx = state
        gates = (torch.mm(input, self.weight_ih.t()) + self.bias_ih + torch.mm(hx, self.weight_hh.t()) + self.bias_hh)
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = torch.sigmoid(ingate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = torch.tanh(cellgate)
        outgate = torch.sigmoid(outgate)

        cy = (forgetgate * cx) + (ingate * cellgate)
        hy = outgate * torch.tanh(cy)

        return hy, cy

# Gated Recurrent Unit
he Gated Recurrent Unit (GRU) is the younger sibling of the more popular Long Short-Term Memory (LSTM) network. GRU outshines its elder sibling due to its superior speed while achieving similar accuracy and effectiveness.


The first thing we need to introduce are reset and update gates. We engineer them to be vectors with entries in  (0,1) such that we can perform convex combinations. The reset variable would allow us to control how much of the previous state we might still want to remember. Likewise, an update variable would allow us to control how much of the new state is just a copy of the old state.  
<font size="4">$R_t = \sigma(W_r . x_t + U_r . H_{t-1} + b_r)$</font>   
<font size="4">$Z_t = \sigma(W_z . x_t + U_z . H_{t-1} + b_z)$</font>  
Where $Z_t$ and $R_t$ represent the **update** and **reset** get respectively of the current cell.  
![GRU update and reset gate](Images/GRU1.svg)  


Next we update the memory
<font size="4">$\tilde{H}_t = tanh(W_h . x_t + r * U_h . H_{t-1} + b_h)$</font>  
While $\tilde{H}_t$ is **intermediate memory** of cell $t$.   
![GRU memory update](Images/GRU2.svg)

Output is generated as follows  
<font size="4">$H_t = Z * H_{t-1} + (1-Z)*\tilde{H}$</font>  
![GRU memory update](Images/GRU3.svg)  
While $H_t$ represent the **output** of the current cell. 

In [7]:
class GRUCell(nn.Module):

    """
    An implementation of GRUCell.

    """

    def __init__(self, input_size, hidden_size, bias=True):
        super(GRUCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 3 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 3 * hidden_size, bias=bias)
        self.reset_parameters()



    def reset_parameters(self):
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)
    
    def forward(self, x, hidden):
        
        x = x.view(-1, x.size(1))
        
        gate_x = self.x2h(x) 
        gate_h = self.h2h(hidden)
        
        gate_x = gate_x.squeeze()
        gate_h = gate_h.squeeze()
        
        i_r, i_i, i_n = gate_x.chunk(3, 1)
        h_r, h_i, h_n = gate_h.chunk(3, 1)
        
        
        resetgate = F.sigmoid(i_r + h_r)
        inputgate = F.sigmoid(i_i + h_i)
        newgate = F.tanh(i_n + (resetgate * h_n))
        
        hy = newgate + inputgate * (hidden - newgate)
        
        
        return hy